![](https://www.ishn.com/ext/resources/Issues/2019/06-June/ISHN0619_C2_pic.jpg?1559575718)

# **Introduction and description of the data and the project**

This project's objective is to use a machine learning and data mining algorithms that predict whether a person in a given county is likely to have a specific infectious disease based on a data from a dataset from Kaggle called [Infectious Disease 2001-2004](https://www.kaggle.com/datasets/jahaidulislam/infectious-disease-2001-2014).

Infectious diseases, caused by pathogens like viruses, bacteria, fungi, and parasites, spread through various means including direct contact, droplets, airborne particles, and contaminated food or water. These diseases significantly impact public health and the global economy, with varying incidence and prevalence across different regions and populations over time.

The dataset tracks outbreaks by year and state, detailing the location, type, duration, and demographic information of affected individuals, such as age, gender, and race. It also includes information on the suspected or confirmed pathogens, modes of transmission, and outbreak sources, covering a range of diseases like foodborne illnesses, respiratory infections, and sexually transmitted infections.

Dataset Features:
*   Disease
*   County
*   Year
*   Sex
*   Count
*   Population
*   Rate
*   CI.lower
*   CI.upper
*   Unstable